In [19]:
import numpy as np

from numpy.testing import assert_allclose


In [20]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    m = np.linalg.norm(vec)
    x = vec.shape[0]
    y = np.zeros(x)
    y[0] = m
    
    u = (vec - y) / np.linalg.norm(vec - y)
    H = np.eye(x) - 2*np.sign(vec[0])*np.outer(u, u)
    
    return y, H


In [33]:
# Test I.1 (10% of the total grade).
v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol=1e-10)
assert_allclose(np.dot(h, v), v1, atol=1e-10)

In [50]:
# Test I.2 (10% of the total grade).
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

In [38]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    H = np.eye(m)
    R = a1.copy()
    for i in range(n): 
        vec = a1[i:, i]
        H_i = np.eye(m)
        H_i[i:, i:] = householder(vec)[1]
        H = H @ H_i
        R = H_i @ R
    return H, R    

In [39]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc
np.set_printoptions(suppress=True)


In [40]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [41]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [68]:
print(q)
print(qq)
print(r)
print(rr)

[[ 0.13665049  0.84904838  0.08201038 -0.29370895 -0.40921002]
 [ 0.56035895  0.01667202  0.71836649 -0.05857033  0.40772416]
 [ 0.19725922  0.3896956  -0.22228119  0.84198854  0.22554936]
 [ 0.62498418 -0.3541743  -0.08592792  0.17378815 -0.66810883]
 [ 0.48765568  0.03920202 -0.64840678 -0.41371192  0.41117666]]
[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]
[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.75489996  0.58665437]
 [ 0.         -0.05978056 -0.26437931]
 [-0.          0.21409591  0.59610183]
 [-0.          0.29828915 -0.03435307]]
[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0

Разложения не совпадают. QR разложение единтсвенно только для вырожденных матриц. Очевидно, библиотечная функция считает qr-разложение по другому алгоритму.

In [69]:
def QR_dec(a):
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    reflect = []
    R = a1.copy()
    for i in range(n):
        vec = R[i:, i]
        x = vec.shape[0]
        y = np.zeros(x)
        y[0] = np.linalg.norm(vec)
        u = (vec - y) / np.linalg.norm(vec - y)
        reflect.append(u)
        R[i:, i:] = R[i:, i:] - 2*np.outer(u,np.dot(u,a1))
    return R, reflect
print(R)
print(reflect)

[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.         -0.        ]]
[array([-0.6570196 ,  0.42644006,  0.15011669,  0.47562065,  0.37111197]), array([-0.52846942,  0.73983285, -0.10990213,  0.40160796]), array([-0.79133207,  0.36468006, -0.49071581])]
